In [1]:
import os
import glob
import json
import quickstats

In [2]:
# make sure hh combination fw path is defined
hh_comb_fw_path = os.environ.get('hh_combination_fw_path', None)
hh_comb_fw_path = "/afs/cern.ch/work/c/chlcheng/Repository/hh_combination_fw"

assert hh_comb_fw_path

## Step 1: Configure task

In [3]:
# specify the input timestamp to use
timestamp = "20220425"

paths = {
    'input'              : os.path.join(hh_comb_fw_path, "FullRun2Workspaces", "original", "HHH2022", timestamp),
    'task_options'       : os.path.join(hh_comb_fw_path, "configs", "task_options"       , "HHH2022",
                                        "nonres_kl_kt_likelihood_v2.yaml"),
    'correlation_schemes': os.path.join(hh_comb_fw_path, "configs", "correlation_schemes", "HHH2022", 
                                        "nonres_kl_v11.json"),
    # specify your output directory
    'output'             : os.path.join(os.getcwd(), f"outputs_HHH2022_{timestamp}")
}

config = {
    'resonant_type': 'nonres',
    'channels'     : ['bbbb', 'bbtautau', 'bbyy'],
    'file_expr'    : '<mass[F]>_kl',
    'blind'        : False,
    'cache'        : True,
    'experimental' : True,
    'parallel'     : -1
}

## Step 2: Prepare modified channel workspaces

In [4]:
options = {
    "input_dir"    : paths['input'],
    "resonant_type": config['resonant_type'],
    "channels"     : ",".join(config['channels']),
    "outdir"       : paths['output'],
    "file_expr"    : f"\"{config['file_expr']}\"",
    "config"       : paths['task_options'],
    "parallel"     : config['parallel'],
    "skip-limit"   : ""
}
if config["blind"]:
    options["blind"] = ""
else:
    options["unblind"] : ""
if config["experimental"]:
    options["experimental"] = ""
else:
    options["official"] : ""
if config['cache']:
    options['cache'] = ""
else:
    options['no-cache'] = ""
command_str = "HHComb process_channels " + " ".join([f"--{key} {value}" for key, value in options.items()])
print(command_str)

HHComb process_channels --input_dir /afs/cern.ch/work/c/chlcheng/Repository/hh_combination_fw/FullRun2Workspaces/original/HHH2022/20220425 --resonant_type nonres --channels bbbb,bbtautau,bbyy --outdir /afs/cern.ch/work/c/chlcheng/Repository/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220425 --file_expr "<mass[F]>_kl" --config /afs/cern.ch/work/c/chlcheng/Repository/hh_combination_fw/configs/task_options/HHH2022/nonres_kl_kt_likelihood_v2.yaml --parallel -1 --skip-limit  --experimental  --cache 


In [5]:
os.system(command_str)

Skip: rescaling output /afs/cern.ch/work/c/chlcheng/Repository/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220425/rescaled/nonres/bbbb/0_kl.root exists, skip rescaling
INFO: Task finished. Total time taken: 0.005 s
Skip: rescaling output /afs/cern.ch/work/c/chlcheng/Repository/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220425/rescaled/nonres/bbtautau/0_kl.root exists, skip rescaling
INFO: Task finished. Total time taken: 0.003 s

RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt

INFO: Writing rescaling log into /afs/cern.ch/work/c/chlcheng/Repository/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220425/rescaled/nonres/bbyy/0_kl.log
INFO: Task finished. Total time taken: 5.119 s


0

## Step 3: Prepare combined workspace

In [6]:
options = {
    "input_dir"    : paths['output'],
    "resonant_type": config['resonant_type'],
    "channels"     : ",".join(config['channels']),
    "file_expr"    : f"\"{config['file_expr']}\"",
    "config"       : paths['task_options'],
    "scheme"       : paths['correlation_schemes'],
    "parallel"     : config['parallel'],
    "skip-limit"   : ""
}
if config["blind"]:
    options["blind"] = ""
else:
    options["unblind"] : ""
if config["experimental"]:
    options["experimental"] = ""
else:
    options["official"] : ""
if config['cache']:
    options['cache'] = ""
else:
    options['no-cache'] = ""
command_str = "HHComb combine_ws " + " ".join([f"--{key} {value}" for key, value in options.items()])
print(command_str)

HHComb combine_ws --input_dir /eos/user/c/chlcheng/analysis/HHComb/H_HH_combination/outputs_HHH2022_20220425 --resonant_type nonres --channels bbbb,bbtautau,bbyy --file_expr "<mass[F]>_kl" --config /afs/cern.ch/work/c/chlcheng/Repository/hh_combination_fw/configs/task_options/HHH2022/nonres_kl_kt_likelihood_v2.yaml --scheme /afs/cern.ch/work/c/chlcheng/Repository/hh_combination_fw/configs/correlation_schemes/HHH2022/nonres_kl_v11.json --parallel -1 --skip-limit  --experimental  --cache 


In [7]:
os.system(command_str)

INFO: Registered the following param points and corresponding channels for combination
(mass=0): ['bbbb', 'bbtautau', 'bbyy']

RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt

INFO: Combination config for the point "mass=0" saved as "/eos/user/c/chlcheng/analysis/HHComb/H_HH_combination/outputs_HHH2022_20220425/cfg/combination/nonres/A-bbbb_bbtautau_bbyy-fullcorr/0_kl.xml"
/afs/cern.ch/work/c/chlcheng/Repository/hh_combination_fw/submodules/workspaceCombiner/build/manager -w combine -x /eos/user/c/chlcheng/analysis/HHComb/H_HH_combination/outputs_HHH2022_20220425/cfg/combination/nonres/A-bbbb_bbtautau_bbyy-fullcorr/0_kl.xml -f /eos/user/c/chlcheng/analysis/HHComb/H_HH_combination/outputs_HHH2022_20220425/combined/nonres/A-bbbb_bbtautau_bbyy-fullcorr/0_kl.root -s 0 -t -1
Skip: combined wor

0

**Checkout the rescaled workspace paths**

In [8]:
rescaled_ws_paths = {}
for channel in config['channels']:
    ws_path = os.path.join(paths['output'], 'rescaled', config['resonant_type'], channel, "0_kl.root")
    rescaled_ws_paths[channel] = ws_path
# rescaled workspace path for combined workspacee
ws_path = glob.glob(os.path.join(paths['output'], 'combined', config['resonant_type'], "*", "0_kl.root"))[0]
rescaled_ws_paths["combined"] = ws_path
rescaled_ws_paths

{'bbbb': '/eos/user/c/chlcheng/analysis/HHComb/H_HH_combination/outputs_HHH2022_20220425/rescaled/nonres/bbbb/0_kl.root',
 'bbtautau': '/eos/user/c/chlcheng/analysis/HHComb/H_HH_combination/outputs_HHH2022_20220425/rescaled/nonres/bbtautau/0_kl.root',
 'bbyy': '/eos/user/c/chlcheng/analysis/HHComb/H_HH_combination/outputs_HHH2022_20220425/rescaled/nonres/bbyy/0_kl.root',
 'combined': '/eos/user/c/chlcheng/analysis/HHComb/H_HH_combination/outputs_HHH2022_20220425/combined/nonres/A-bbbb_bbtautau_bbyy-fullcorr/0_kl.root'}

## Step 4: Prepare Asimov workspaces

**Set up the asimov workspace paths**

In [9]:
asimov_ws_paths = {}
for channel in rescaled_ws_paths:
    rescaled_ws_path = rescaled_ws_paths[channel]
    asimov_ws_path   = os.path.join(os.path.dirname(rescaled_ws_path), "0_asimov.root")
    asimov_ws_paths[channel] = asimov_ws_path
asimov_ws_paths

{'bbbb': '/eos/user/c/chlcheng/analysis/HHComb/H_HH_combination/outputs_HHH2022_20220425/rescaled/nonres/bbbb/0_asimov.root',
 'bbtautau': '/eos/user/c/chlcheng/analysis/HHComb/H_HH_combination/outputs_HHH2022_20220425/rescaled/nonres/bbtautau/0_asimov.root',
 'bbyy': '/eos/user/c/chlcheng/analysis/HHComb/H_HH_combination/outputs_HHH2022_20220425/rescaled/nonres/bbyy/0_asimov.root',
 'combined': '/eos/user/c/chlcheng/analysis/HHComb/H_HH_combination/outputs_HHH2022_20220425/combined/nonres/A-bbbb_bbtautau_bbyy-fullcorr/0_asimov.root'}

**Generate CLI commands**

In [10]:
options = {
    'poi'         : 'xsec_br',
    'data'        : 'combData',
    'asimov_types': '2'
}
command_str_map = {}
for channel in rescaled_ws_paths:
    input_file  = rescaled_ws_paths[channel]
    output_file = asimov_ws_paths[channel]
    channel_options = {"input_file": input_file, "output_file": output_file, **options}
    channel_command_str = "quickstats generate_standard_asimov " + \
                          " ".join([f"--{key} {value}" for key, value in channel_options.items()])
    command_str_map[channel] = channel_command_str
    print(channel_command_str)
    print()

quickstats generate_standard_asimov --input_file /eos/user/c/chlcheng/analysis/HHComb/H_HH_combination/outputs_HHH2022_20220425/rescaled/nonres/bbbb/0_kl.root --output_file /eos/user/c/chlcheng/analysis/HHComb/H_HH_combination/outputs_HHH2022_20220425/rescaled/nonres/bbbb/0_asimov.root --poi xsec_br --data combData --asimov_types 2

quickstats generate_standard_asimov --input_file /eos/user/c/chlcheng/analysis/HHComb/H_HH_combination/outputs_HHH2022_20220425/rescaled/nonres/bbtautau/0_kl.root --output_file /eos/user/c/chlcheng/analysis/HHComb/H_HH_combination/outputs_HHH2022_20220425/rescaled/nonres/bbtautau/0_asimov.root --poi xsec_br --data combData --asimov_types 2

quickstats generate_standard_asimov --input_file /eos/user/c/chlcheng/analysis/HHComb/H_HH_combination/outputs_HHH2022_20220425/rescaled/nonres/bbyy/0_kl.root --output_file /eos/user/c/chlcheng/analysis/HHComb/H_HH_combination/outputs_HHH2022_20220425/rescaled/nonres/bbyy/0_asimov.root --poi xsec_br --data combData --asi

**Generate asimov data**

In [11]:
for channel, command_str in command_str_map.items():
    print(f"INFO: Generating asimov data for the channel \"{channel}\"")
    os.system(command_str)

INFO: Generating asimov data for the channel "bbbb"

RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt

INFO: Loaded extension module "FlexibleInterpVarMkII"
INFO: Opening file "/eos/user/c/chlcheng/analysis/HHComb/H_HH_combination/outputs_HHH2022_20220425/rescaled/nonres/bbbb/0_kl.root"
INFO: Loaded workspace "combWS"
INFO: Loaded model config "ModelConfig"
INFO: Activated binned likelihood attribute for Region_distmHH_L0_Y2016_T4_J4_BMin0_DXhhIndEtaHi_model
INFO: Activated binned likelihood attribute for Region_distmHH_L0_Y2016_T4_J4_BMin0_DXhhIndEtaLo_model
INFO: Activated binned likelihood attribute for Region_distmHH_L0_Y2016_T4_J4_BMin0_DXhhIndEtaMi_model
INFO: Activated binned likelihood attribute for Region_distmHH_L0_Y2016_T4_J4_BMin0_DXhhOudEtaHi_model
INFO: Activated binned likel

## Step 5: Prepare post-fit snapshots

**Set up the fitted workspace paths**

In [12]:
fitted_ws_paths = {}
for channel in asimov_ws_paths:
    asimov_ws_path = asimov_ws_paths[channel]
    fitted_ws_path = os.path.join(os.path.dirname(asimov_ws_path), "0_fitted.root")
    fitted_ws_paths[channel] = fitted_ws_path
fitted_ws_paths

{'bbbb': '/eos/user/c/chlcheng/analysis/HHComb/H_HH_combination/outputs_HHH2022_20220425/rescaled/nonres/bbbb/0_fitted.root',
 'bbtautau': '/eos/user/c/chlcheng/analysis/HHComb/H_HH_combination/outputs_HHH2022_20220425/rescaled/nonres/bbtautau/0_fitted.root',
 'bbyy': '/eos/user/c/chlcheng/analysis/HHComb/H_HH_combination/outputs_HHH2022_20220425/rescaled/nonres/bbyy/0_fitted.root',
 'combined': '/eos/user/c/chlcheng/analysis/HHComb/H_HH_combination/outputs_HHH2022_20220425/combined/nonres/A-bbbb_bbtautau_bbyy-fullcorr/0_fitted.root'}

In [13]:
pois_to_scan = ["klambda", "kt", "kV", "k2V", "kF", "kH", "ktau", "klambda,kt", "kV,k2V"]
minimizer_options = {
    "retry": 2,
    "eps": 1
}
# observed and expected datasets
datasets = {
    "observed": "combData",
    "expected": "asimovData_muhat_NP_Profile"
}

In [14]:
from quickstats.components import AnalysisBase

Welcome to JupyROOT 6.24/06

RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt



In [15]:
for channel in fitted_ws_paths:
    print(f"==> Channel: {channel}")
    asimov_ws_path = asimov_ws_paths[channel]
    fitted_ws_path = fitted_ws_paths[channel]
    kwargs = {
        "filename" : asimov_ws_path,
        # only need to generate snapshot for observed data
        "data_name": datasets["observed"],
        "poi_name": [],
        "config": {**minimizer_options}
    }
    analysis = AnalysisBase(**kwargs)
    # fix all pois at the beginning
    analysis.setup_parameters(fix_param="<pois>")
    analysis.save_snapshot(analysis.kTempSnapshotName)
    for poi_expr in pois_to_scan:
        print(f"==> POIs: {poi_expr}")
        analysis.setup_parameters(profile_param=poi_expr)
        analysis.nll_fit(mode=3)
        pois = poi_expr.split(",")
        snapshot_name = f"obs_bestfit_{'_'.join(pois)}"
        # save best-fit snapshot
        analysis.save_snapshot(snapshot_name)
        analysis.load_snapshot(analysis.kTempSnapshotName)
    # restore initial snapshot
    analysis.load_snapshot(analysis.kInitialSnapshotName)
    analysis.save(fitted_ws_path)

==> Channel: bbbb
INFO: Loaded extension module "FlexibleInterpVarMkII"
INFO: Opening file "/eos/user/c/chlcheng/analysis/HHComb/H_HH_combination/outputs_HHH2022_20220425/rescaled/nonres/bbbb/0_asimov.root"
INFO: Loaded workspace "combWS"
INFO: Loaded model config "ModelConfig"
INFO: Activated binned likelihood attribute for Region_distmHH_L0_Y2016_T4_J4_BMin0_DXhhIndEtaHi_model
INFO: Activated binned likelihood attribute for Region_distmHH_L0_Y2016_T4_J4_BMin0_DXhhIndEtaLo_model
INFO: Activated binned likelihood attribute for Region_distmHH_L0_Y2016_T4_J4_BMin0_DXhhIndEtaMi_model
INFO: Activated binned likelihood attribute for Region_distmHH_L0_Y2016_T4_J4_BMin0_DXhhOudEtaHi_model
INFO: Activated binned likelihood attribute for Region_distmHH_L0_Y2016_T4_J4_BMin0_DXhhOudEtaLo_model
INFO: Activated binned likelihood attribute for Region_distmHH_L0_Y2016_T4_J4_BMin0_DXhhOudEtaMi_model
INFO: Activated binned likelihood attribute for Region_distmHH_L0_Y2017_T4_J4_BMin0_DXhhIndEtaHi_model


INFO: NLL evaluation completed with
     prefit NLL = 292290.38600
    postfit NLL = 290961.60291
           time = 55.765
==> POIs: ktau
INFO: Float parameter "ktau" at value 1.0 [-1e+30, 1e+30]
INFO: NLL evaluation completed with
     prefit NLL = 292290.38600
    postfit NLL = 290961.53794
           time = 54.285
==> POIs: klambda,kt
INFO: Float parameter "klambda" at value 1.0 [-1e+30, 1e+30]
INFO: Float parameter "kt" at value 1.0 [-1e+30, 1e+30]
INFO: NLL evaluation completed with
     prefit NLL = 292290.38600
    postfit NLL = 290961.54187
           time = 55.487
==> POIs: kV,k2V
INFO: Float parameter "kV" at value 1.0 [-1e+30, 1e+30]
INFO: Float parameter "k2V" at value 1.0 [-1e+30, 1e+30]
INFO: NLL evaluation completed with
     prefit NLL = 292290.38600
    postfit NLL = 290961.53518
           time = 58.277
INFO: Saved workspace file as `/eos/user/c/chlcheng/analysis/HHComb/H_HH_combination/outputs_HHH2022_20220425/rescaled/nonres/bbtautau/0_fitted.root`
==> Channel: bbyy

INFO: NLL evaluation completed with
     prefit NLL = 758381.90963
    postfit NLL = 756982.06623
           time = 104.446
==> POIs: kV
INFO: Float parameter "kV" at value 1.0 [-1e+30, 1e+30]
INFO: NLL evaluation completed with
     prefit NLL = 758381.90963
    postfit NLL = 756982.91593
           time = 43.110
==> POIs: k2V
INFO: Float parameter "k2V" at value 1.0 [-1e+30, 1e+30]
INFO: NLL evaluation completed with
     prefit NLL = 758381.90963
    postfit NLL = 756983.03557
           time = 44.604
==> POIs: kF
INFO: Float parameter "kF" at value 1.0 [-1e+30, 1e+30]
INFO: NLL evaluation completed with
     prefit NLL = 758381.90963
    postfit NLL = 756983.03490
           time = 47.254
==> POIs: kH
INFO: Float parameter "kH" at value 1.0 [-1e+30, 1e+30]
INFO: NLL evaluation completed with
     prefit NLL = 758381.90963
    postfit NLL = 756982.93756
           time = 114.230
==> POIs: ktau
INFO: Float parameter "ktau" at value 1.0 [-1e+30, 1e+30]
INFO: NLL evaluation completed w

- By now, you should have the workspace `0_fitted.root` for each channel
- Inside `0_fitted.root`, you have the observed dataset `combData` and the asimov dataset `asimovData_muhat_NP_Profile`
- For each dataset, there are the corresponding muhat snapshots designed for each likelihood scan, e.g.
  - obs_bestfit_klambda, obs_bestfit_kt, obs_bestfit_kV, obs_bestfit_k2V, obs_bestfit_klambda_kt, obs_bestfit_kV_k2V
  - exp_bestfit_klambda, exp_bestfit_kt, exp_bestfit_kV, exp_bestfit_k2V, exp_bestfit_klambda_kt, exp_bestfit_kV_k2V

## Step 6: Run likelihood scans

**Only commands are shown here since the jobs are computationally intensive**

In [16]:
param_expr_maps = {
    'klambda': "klambda=-6_12_0.2",
    "klambda_kt_profile": "klambda=-16_20_0.4,kt",
    "klambda_kt_2D": "klambda=-15_20_0.2,kt=0.6_1.6_0.1",
    "kt": "kt=-2_6_0.1",
    "kF": "kF=-8_15_0.1",
    "kV": "kV=-2.5_2.5_0.1",
    "k2V": "k2V=-3_5_0.1",
    "kV_k2V_2D": "kV=-3_3_0.2,k2V=-4_10_0.2"
}

In [17]:
from quickstats.parsers import ParamParser

In [18]:
likelihood_scan_cmds = {}
for channel in fitted_ws_paths:
    likelihood_scan_cmds[channel] = {}
    fitted_ws_path = fitted_ws_paths[channel]
    for dataset_type, dataset_name in datasets.items():
        likelihood_scan_cmds[channel][dataset_type] = {}
        for key, expr in param_expr_maps.items():
            outdir = os.path.join(paths['output'], 'likelihood', dataset_type, channel, key)
            pois = ParamParser._get_param_str_attributes(expr)
            if dataset_type == "observed":
                snapshot_name = f"obs_bestfit_{'_'.join(pois)}"
            else:
                snapshot_name = dataset_name
            options = {
                "input_file" : fitted_ws_path,
                "data": dataset_name,
                "param_expr": expr,
                'outdir': outdir,
                'snapshot': snapshot_name,
                **minimizer_options
            }
            cmd_str = "quickstats likelihood_scan " + \
                      " ".join([f"--{key} {value}" for key, value in options.items()])
            likelihood_scan_cmds[channel][dataset_type][key] = cmd_str

In [19]:
for channel in likelihood_scan_cmds:
    print("#########################################################################################")
    print(f"# Channel: {channel}")
    for dataset_type in likelihood_scan_cmds[channel]:
        print(f"# Dataset: {dataset_type}")
        for key in likelihood_scan_cmds[channel][dataset_type]:
            print(f"# Scan parameters: {key}")
            cmd_str = likelihood_scan_cmds[channel][dataset_type][key]
            print(cmd_str)
        print()
    print("#########################################################################################")

#########################################################################################
# Channel: bbbb
# Dataset: observed
# Scan parameters: klambda
quickstats likelihood_scan --input_file /eos/user/c/chlcheng/analysis/HHComb/H_HH_combination/outputs_HHH2022_20220425/rescaled/nonres/bbbb/0_fitted.root --data combData --param_expr klambda=-6_12_0.2 --outdir /eos/user/c/chlcheng/analysis/HHComb/H_HH_combination/outputs_HHH2022_20220425/likelihood/observed/bbbb/klambda --snapshot obs_bestfit_klambda --retry 2 --eps 1
# Scan parameters: klambda_kt_profile
quickstats likelihood_scan --input_file /eos/user/c/chlcheng/analysis/HHComb/H_HH_combination/outputs_HHH2022_20220425/rescaled/nonres/bbbb/0_fitted.root --data combData --param_expr klambda=-16_20_0.4,kt --outdir /eos/user/c/chlcheng/analysis/HHComb/H_HH_combination/outputs_HHH2022_20220425/likelihood/observed/bbbb/klambda_kt_profile --snapshot obs_bestfit_klambda_kt --retry 2 --eps 1
# Scan parameters: klambda_kt_2D
quickstats likel